# Packages

In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Utils
import time
import os
loc = os.getcwd() 
import sys 

# Matplotlib Params
import matplotlib
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 16}

matplotlib.rc('font', **font)


# Tensorflow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, confusion_matrix


# Oversampling 
from imblearn.over_sampling import RandomOverSampler

from main_feats import featExtract

from sklearn import svm
from sklearn.metrics import f1_score, roc_auc_score, roc_curve


# Introduction

The features that are computed by featExtract are: 
- Spectral power (pow)
- Spectral entropy (ent)
- Spectral centroid (cent)
- AM, FM bandwidths (AM, BM)
- Hjorth Mobility (Hmob)
- Hjorth Complexity (Hcomp)
- Skew, Kurtosis

The formulas can be found at 
(link paper)

# Auxiliary Functions 

In [19]:
def featExt(data): # for 1600 parameter
    '''
    featExtract extracts '''
    return(featExtract(data, Fs=1600)) 

def unpack_tuple(feats):
    '''
    Input: sens1 is a panda series of outputs from applying featExtract to another series
    Output: panda dataframe with the correct column names and output'''
    # idea: loop through series, unpack tuple, create columns
    _, col_names = feats.iloc[0]
    df = pd.DataFrame(columns=col_names)
    feats_first = feats.first_valid_index()
    for idx, tuples in enumerate(feats):
        data, _ = tuples
        df.loc[idx+feats_first] = data
    return df

def log_cols(df, cols):
    for col in cols:
        df[col] = df[col].apply(lambda a: np.log(a))



# Load and Process Data

In [24]:
# load in the data 
data_dict = np.load('/Users/wang_to/Documents/University/Anomaly_detection/anomalies/bookshelf/dataset/data_dict.npy',allow_pickle='TRUE').item()
# change this relative file

In [61]:
num_sens = 24
sens = []

for i in range(num_sens):
    sens.append(data_dict[f"Sensor{i+1}"])

sens = np.concatenate(sens)

sens_data = sens[:,:-4].astype('float32') # actual time series

sens = np.where(sens == 'D00', 0, sens) # find and replace: encode different damage levels
sens = np.where(sens == 'DB0', 1, sens)
sens = np.where(sens == 'DBB', 2, sens)
sens = np.where(sens == 'DHT', 3, sens)
sens = np.where(sens == 'D05', 4, sens)
sens = np.where(sens == 'D10', 5, sens)

sens = pd.DataFrame(sens)
sens_labels = sens.iloc[:, -4] # 0 and 1 labels for whether damaged or not
sens_damage_levels = sens.iloc[:, -2] # 0, 1, 2, 3, 4, 5 labels for different kinds of damage

sens.columns = data_dict['Sensor1'].columns

In [83]:
data = sens.iloc[:,:8192].astype('float32')
features_data = data.apply(featExt, axis=1)
healthy_input = pd.concat([features_data, sens["voltage_num"]], axis=1)


In [99]:
healthy_input = unpack_tuple(features_data)

healthy_input["AM"] = healthy_input["AM"].apply(lambda a: np.log(a))
healthy_input["BM"] = healthy_input["BM"].apply(lambda a: np.log(a))
healthy_input["pow"] = healthy_input["pow"].apply(lambda a: np.log(a))

# concatenate voltage back on as a feature, and 
healthy_input = pd.concat([healthy_input, sens["voltage_num"]], axis=1)
healthy_input.columns = [*healthy_input.columns[:-1], 'voltage']

scaler = MinMaxScaler()
healthy_input.iloc[:,:11] = scaler.fit_transform(healthy_input.iloc[:,:11]) 


In [171]:
len(healthy_input.columns)

12

In [100]:
healthy_input

,AM,BM,ent,pow,Cent,pk,freq,skew,kurt,Hmob,Hcomp,voltage
0,14.770663,18.219263,5.434050,-25.682845,179.110836,0.000046,32.8125,0.019741,-0.237593,0.920375,1.908907,2
1,14.502801,18.051682,5.090054,-25.073920,155.171930,0.000052,32.8125,-0.043976,0.184551,0.781151,2.214398,2
2,14.127033,17.728356,4.110170,-23.553807,105.560881,0.000129,32.8125,-0.021230,-0.381595,0.627150,2.761172,2
3,14.362882,17.899392,4.344860,-23.908486,128.602243,0.000110,59.3750,0.016864,-0.363946,0.728563,2.390157,2
4,14.230340,17.890359,4.251189,-23.522809,119.429701,0.000145,59.3750,-0.041615,-0.125535,0.692293,2.495475,2
...,...,...,...,...,...,...,...,...,...,...,...,...
6475,11.693476,17.781814,2.924750,-13.750972,98.013310,0.020166,59.3750,-0.007092,-0.105375,0.536828,3.098347,8
6476,12.216511,17.916929,3.507044,-15.017141,123.854526,0.010189,59.3750,-0.018624,-0.443364,0.672694,2.589241,8
6477,12.102453,17.940982,3.626921,-15.041137,122.806889,0.010689,59.3750,0.029003,-0.406746,0.639949,2.647436,8
6478,11.892609,17.845560,3.361149,-14.416121,107.356277,0.014414,59.3750,0.039151,-0.555301,0.575893,2.905574,8


In [107]:
sens

,Xt_0,Xt_1,Xt_2,Xt_3,Xt_4,Xt_5,Xt_6,Xt_7,Xt_8,Xt_9,...,Xt_8187,Xt_8188,Xt_8189,Xt_8190,Xt_8191,y_labels,damage_location,damage_level,voltage_level,voltage_num
0,0.015912,0.034565,0.00965,0.018976,0.007931,0.027672,0.022788,0.014636,0.016848,0.00708,...,0.021511,0.04966,-0.003863,0.035092,0.039466,0,L00,0,V02,2
1,-0.016916,0.004459,0.005973,-0.021001,0.007846,-0.008424,-0.0129,-0.004459,-0.014006,-0.017767,...,-0.003063,0.008918,-0.015266,-0.020797,-0.012866,0,L00,0,V02,2
2,0.038836,0.038173,0.03048,0.033373,0.060467,0.016491,0.033305,0.039023,0.00594,0.031246,...,0.041491,0.052655,0.003029,0.010688,0.004152,0,L00,0,V02,2
3,0.001549,-0.020507,-0.023281,-0.01147,-0.023128,-0.013649,-0.002927,-0.021971,-0.003591,-0.000987,...,-0.004102,-0.01804,0.026192,0.00291,0.003846,0,L00,0,V02,2
4,0.012764,0.014755,0.026294,0.0153,0.004493,0.007284,0.000579,-0.018959,-0.007658,-0.040045,...,0.004816,-0.011777,0.008288,-0.009054,0.024881,0,L00,0,V02,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6475,-0.24319,-0.19547,-0.39135,-0.15181,-0.20088,-0.20791,-0.032714,-0.008516,0.18223,0.21062,...,0.20102,0.27483,0.24698,0.20264,0.20656,1,L3A,2,V08,8
6476,0.10801,0.11896,0.40785,0.37594,0.43867,0.27983,0.36175,0.2732,0.3485,0.19858,...,-0.11464,-0.089086,0.071917,0.089086,0.13762,1,L3A,2,V08,8
6477,0.39527,0.40501,0.14911,0.33498,0.084084,0.089626,0.15397,0.045016,0.024738,0.083138,...,0.016628,-0.13924,-0.1975,-0.26699,-0.27307,1,L3A,2,V08,8
6478,0.32065,0.47071,0.45962,0.4311,0.43921,0.40879,0.19885,0.18763,-0.025009,-0.077054,...,0.098954,0.1414,0.12423,0.20277,0.2048,1,L3A,2,V08,8


In [165]:
train_X = np.array(healthy_input)
train_Y = sens['damage_level'].to_numpy().flatten().astype('int')

# SVM Models
## SVM with Multiclass Classification
The code below implements support vector machines (linear and RBF kernels) with the above training data. The models classify each input into one of 6 damage levels (encoded 0, 1, 2, 3, 4, 5 denoting D00, DB0, DBB, DHT, D05, D10 respectively). 

In [170]:
num_folds = 5
kfold = KFold(n_splits = num_folds, shuffle=True, random_state=1337)
kfold.get_n_splits(train_X)
fold_no = 1 

linear_scores_per_fold = []
rbf_scores_per_fold = []
scores_per_fold = []

for C in [1, 10, 100, 1000]:
    fold_no = 0
    for train_index, test_index in kfold.split(train_X):
        print(f'Fold no: {fold_no}')
        X_train, X_test = train_X[train_index], train_X[test_index]
        Y_train, Y_test = train_Y[train_index], train_Y[test_index]

        linear_svm = svm.SVC(C=C, kernel="linear", gamma = "auto")
        linear_svm.fit(train_X, train_Y)
        linear_new_score = linear_svm.score(X_test, Y_test)
        linear_scores_per_fold.append(linear_new_score)

        print(f'Linear accuracy for fold {fold_no}: {linear_new_score}.')

        rbf_svm = svm.SVC(C=C, kernel="rbf", gamma = "auto")
        rbf_svm.fit(train_X, train_Y)
        rbf_new_score = rbf_svm.score(X_test, Y_test)
        rbf_scores_per_fold.append(rbf_new_score)

        print(f'Radial basis function accuracy for fold {fold_no}: {rbf_new_score}.')

        fold_no += 1
    print(f'-----Mean performances for C = {C} ----- \n Linear: {np.mean(linear_new_score)} \n RBF: {np.mean(rbf_scores_per_fold)}')
    linear_scores_per_fold.clear()
    rbf_scores_per_fold.clear()

Fold no: 0
Linear accuracy for fold 0: 0.5794753086419753.
Radial basis function accuracy for fold 0: 0.5717592592592593.
Fold no: 1
Linear accuracy for fold 1: 0.5802469135802469.
Radial basis function accuracy for fold 1: 0.5733024691358025.
Fold no: 2
Linear accuracy for fold 2: 0.5547839506172839.
Radial basis function accuracy for fold 2: 0.5470679012345679.
Fold no: 3
Linear accuracy for fold 3: 0.5671296296296297.
Radial basis function accuracy for fold 3: 0.5640432098765432.
Fold no: 4
Linear accuracy for fold 4: 0.5833333333333334.
Radial basis function accuracy for fold 4: 0.5717592592592593.
-----Mean performances for C = 1 ----- 
 Linear: 0.5833333333333334 
 RBF: 0.5655864197530864
Fold no: 0
Linear accuracy for fold 0: 0.5817901234567902.
Radial basis function accuracy for fold 0: 0.5964506172839507.
Fold no: 1
Linear accuracy for fold 1: 0.5817901234567902.
Radial basis function accuracy for fold 1: 0.5895061728395061.
Fold no: 2
Linear accuracy for fold 2: 0.55941358024

We see quite poor performance, averaging between 55% to 60% accuracy during cross-validation. The engineered features (which work very well in ECG anomaly detection and others) are not able to separate the inputs. We can view the confusion matrix:  

In [169]:
confusion_matrix(rbf_svm.predict(X_test), Y_test)

array([[711, 184, 126,  14,  30,  37],
       [  4,  14,  10,   1,   2,   0],
       [ 11,   9,  46,   1,   1,   3],
       [  3,   8,   7,  41,   1,   0],
       [  0,   2,   1,   0,  15,   2],
       [  2,   0,   3,   0,   0,   7]])

...from which we see that a large number of healthy inputs are being misclassified into the first two levels of damage; the features we have engineered on our data is unable to capture the essence of "healthy" data.

## SVM with Binary Classification

In [162]:
train_Y = sens['y_labels'].to_numpy().flatten().astype('float32')

num_folds = 5
kfold = KFold(n_splits = num_folds, shuffle=True, random_state=1337)
kfold.get_n_splits(train_X)
fold_no = 1 

for C in [1, 10, 100, 1000]:
    fold_no = 0
    for train_index, test_index in kfold.split(train_X):
        print(f'Fold no: {fold_no}')
        X_train, X_test = train_X[train_index], train_X[test_index]
        Y_train, Y_test = train_Y[train_index], train_Y[test_index]

        linear_svm = svm.SVC(C=C, kernel="linear", gamma = "auto")
        linear_svm.fit(train_X, train_Y)
        linear_new_score = linear_svm.score(X_test, Y_test)
        linear_scores_per_fold.append(linear_new_score)

        print(f'Linear accuracy for fold {fold_no}: {linear_new_score}.')

        rbf_svm = svm.SVC(C=C, kernel="rbf", gamma = "auto")
        rbf_svm.fit(train_X, train_Y)
        rbf_new_score = rbf_svm.score(X_test, Y_test)
        rbf_scores_per_fold.append(rbf_new_score)

        print(f'Radial basis function accuracy for fold {fold_no}: {rbf_new_score}.')

        fold_no += 1

    print(f'-----Mean performances----- \n Linear: {np.mean(linear_new_score)} \n RBF: {np.mean(rbf_scores_per_fold)}')
    linear_scores_per_fold.clear()
    rbf_scores_per_fold.clear()


Fold no: 1
Linear accuracy for fold 1: 0.6712962962962963.
Radial basis function accuracy for fold 1: 0.7685185185185185.
Fold no: 2
Linear accuracy for fold 2: 0.6766975308641975.
Radial basis function accuracy for fold 2: 0.7731481481481481.
Fold no: 3
Linear accuracy for fold 3: 0.6705246913580247.
Radial basis function accuracy for fold 3: 0.7646604938271605.
Fold no: 4
Linear accuracy for fold 4: 0.6782407407407407.
Radial basis function accuracy for fold 4: 0.7577160493827161.
Fold no: 5
Linear accuracy for fold 5: 0.6658950617283951.
Radial basis function accuracy for fold 5: 0.7507716049382716.


Numerically-speaking, the performance improves (this is a much simpler problem, after all), although still not ideal: healthy and damaged data can be separated by eye, yet the 12 features we have chosen here do not capture this difference numerically, despite a wide variety of statistical and signal processing-specific features. Feature extraction for noisy sensor data is difficult! 